# 2243

I think I will choose reference ID 2243 (AB1IP_HUMAN, also known as RIAM) as an example where the alignment is particularly problematic

In [3]:
import local_seqtools.general_utils as tools
import pandas as pd
import numpy as np
import local_conservation_analysis_pipeline.group_conservation_objects as group_tools
import json
import copy
import os
import re
import sys
from pathlib import Path
from Bio import AlignIO, Seq, SeqIO, Align

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
def filepath_converter(filepath):
    return filepath.replace('/home/jch/Documents/07-pt1_pt2_combined', '../../..')

In [7]:
aln_file = "./ena_motif_alignments/2243_AB1IP_HUMAN_9606_0:00294e/9606_0_00294e_Vertebrata_1587055at7742_clustered_ldos_aln.fasta"
table = pd.read_csv("./enah_table.csv")
og = group_tools.ConserGene(table.loc[table['reference_index'] == 2243]['json_file'].values[0], filepath_converter=filepath_converter)
lvl = og.get_level_obj("Vertebrata", filepath_converter=filepath_converter)
x = tools.sort_aln_by_pid2ref(lvl.aln, lvl.query_aln_sequence)
subsampled_aln_list = tools.subsample_seqrec_list_target_num(x, target_num=20)
with open(f'../../../../11-pairk/pairk/examples/example_alignment_{og.query_gene_id.replace(":", "_")}-idr-{lvl.idr_aln_start}_to_{lvl.idr_aln_end}.fasta', 'w') as f:
    SeqIO.write(subsampled_aln_list, f, 'fasta')

In [19]:
aln_file = "./ena_motif_alignments/2243_AB1IP_HUMAN_9606_0:00294e/9606_0_00294e_Vertebrata_1587055at7742_clustered_ldos_aln.fasta"
table = pd.read_csv("./enah_table.csv")
og = group_tools.ConserGene(table.loc[table['reference_index'] == 2243]['json_file'].values[0])
lvl = og.get_level_obj("Vertebrata")
x = lvl.aln[:,lvl.hit_aln_start - 30:lvl.hit_aln_end + 30]
query_aln= lvl.query_aln_sequence[lvl.hit_aln_start - 30:lvl.hit_aln_end + 30]
x = tools.sort_aln_by_pid2ref(x, query_aln)
subsampled_list=tools.subsample_seqrec_list_target_num(x, target_num=20)
subsampled_id_list = [x.id for x in subsampled_list]
aln_dict = {s.id: s for s in lvl.aln}
subsampled_fl_aln = [aln_dict[i] for i in subsampled_id_list]
with open('./2243_subsampled_aln_Vertebrata.fasta', 'w') as f:
    SeqIO.write(subsampled_fl_aln, f, 'fasta')

# 2246

In [20]:
aln_file = "./ena_motif_alignments/2246_LPP_HUMAN_9606_0:000d90/9606_0_000d90_Metazoa_124801at33208_clustered_ldos_aln.fasta"
table = pd.read_csv("./enah_table.csv")
og = group_tools.ConserGene(table.loc[table['reference_index'] == 2246]['json_file'].values[0])
lvl = og.get_level_obj("Metazoa")
# x = lvl.aln[:,lvl.hit_aln_start - 30:lvl.hit_aln_end + 30]
# query_aln= lvl.query_aln_sequence[lvl.hit_aln_start - 30:lvl.hit_aln_end + 30]
# x = tools.sort_aln_by_pid2ref(x, query_aln)
# sort the full alignment
x = tools.sort_aln_by_pid2ref(lvl.aln, lvl.query_aln_sequence)
subsampled_list=tools.subsample_seqrec_list_target_num(x, target_num=20)
subsampled_id_list = [x.id for x in subsampled_list]
aln_dict = {s.id: s for s in lvl.aln}
subsampled_fl_aln = [aln_dict[i] for i in subsampled_id_list]
with open('./2246_subsampled_aln_metazoa.fasta', 'w') as f:
    SeqIO.write(subsampled_fl_aln, f, 'fasta')

In [33]:
aln_n = copy.deepcopy(lvl.aln[:,0:lvl.idr_aln_end+1])
width = aln_n.get_alignment_length()
aln_c = lvl.aln[:,lvl.idr_aln_end+1:]
unaln_n = tools.strip_dashes_from_sequences(aln_n)
new_aln_n = []
for seq in unaln_n:
    padded_seq = str(seq.seq).rjust(width, '-')
    new_aln_n.append(tools.str2seqrecord(padded_seq, seq.id))
new_aln_n = AlignIO.MultipleSeqAlignment(new_aln_n)
new_aln = new_aln_n + aln_c
with open('./2246_subsampled_aln_metazoa_padded_stripped_idr.fasta', 'w') as f:
    SeqIO.write(new_aln, f, 'fasta')

In [30]:
padded_seq

'---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [27]:
test="abcd"
print(test[0:1+1])
print(test[1+1:])

ab
cd


In [25]:
test = "AAA"
test.rjust(10, '-')
# pad the test sequence with '-' to the left


'-------AAA'